In [ ]:
!pip install torch torchvision Pillow -q

In [ ]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.0 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.figure import Figure
from pathlib import Path
import random
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLO11 instance segmentation model
model = YOLO("yolo11n-seg.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
results = model.train(
    data="/content/drive/MyDrive/prolog_500.v1i.yolov11/data.yaml",
    epochs=100,

    imgsz=640,
    batch=16,
    workers=2,

    # Гиперпараметры
    lr0=0.01,                 # начальный learning rate
    lrf=0.01,                 # конечный learning rate
    momentum=0.937,           # момент SGD
    weight_decay=0.0005,      # весовая регуляризация
    warmup_epochs=3,          # прогрев
    warmup_momentum=0.8,      # момент при прогреве
    warmup_bias_lr=0.1,       # lr для bias при прогреве


    patience=20,              # ранняя остановка если нет улучшений

    # Оптимизация
    optimizer="SGD",          # можно "AdamW" для маленького датасета
    cos_lr=True,              # косинусный scheduler
    close_mosaic=10,          # отключить mosaic за 10 эпох до конца

    # Валидация
    val=True,                 # валидация после каждой эпохи
    save_period=10,           # сохранять чекпоинты каждые 10 эпох

    # Другое
    device="cuda" if torch.cuda.is_available() else "cpu",
    seed=42,                  # для воспроизводимости
    deterministic=True,       # детерминированные алгоритмы
    single_cls=False,         # мультиклассовая классификация
    plots=True,               # строить графики
    verbose=True,             # подробный вывод
)

model.save("prolog_500_model.pt")

model.save("/content/drive/MyDrive/prolog_500_model.pt")
print("✅ Модель сохранена в Google Drive: MyDrive/prolog_500_model.pt")

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/prolog_500.v1i.yolov11/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=20, perspecti

In [ ]:
# Валидация модели
metrics = model.val(data="/content/drive/MyDrive/prolog_500.v1i.yolov11/data.yaml")

# У объекта SegmentMetrics НЕТ best_map
# Используйте правильные атрибуты:
print("Метрики детекции:")
print(f"  mAP50: {metrics.box.map50:.4f}")
print(f"  mAP50-95: {metrics.box.map:.4f}")
print(f"  Precision: {np.mean(metrics.box.p):.4f}")

print(f"  Recall: {np.mean(metrics.box.r):.4f}")

print("\nМетрики сегментации:")
print(f"  Mask mAP50: {metrics.seg.map50:.4f}")
print(f"  Mask mAP50-95: {metrics.seg.map:.4f}")

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.5±0.2 ms, read: 25.1±6.5 MB/s, size: 46.3 KB)
val: Scanning /content/drive/MyDrive/prolog_500.v1i.yolov11/valid/labels.cache... 149 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 149/149 201.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 2.0it/s 5.0s
                   all        149        154      0.977      0.991      0.985      0.816      0.977      0.991      0.985      0.864
                 crack         39         39      0.974          1      0.988      0.824      0.974          1      0.988      0.887
                 joint        113        115       0.98      0.983      0.983      0.809       0.98      0.983      0.983      0.842
Speed: 4.2ms preprocess, 5.8ms inference, 0.0ms loss, 4.7ms postprocess per image
Results saved to /conte

In [ ]:
source_dir = "/content/runs/segment"
destination_base = "/content/drive/MyDrive/prolog_500.v1i.yolov11/runs"

# Копируем всю папку
shutil.copytree(source_dir, destination_base)

'/content/drive/MyDrive/prolog_500.v1i.yolov11/runs'

In [ ]:
test_folder = "/content/drive/MyDrive/prolog_500.v1i.yolov11/test"
output_dir = "/content/drive/MyDrive/prolog_500.v1i.yolov11/test_predictions"

os.makedirs(output_dir, exist_ok=True)

results = model.predict(
    source=test_folder,
    conf=0.5,
    save=True,
    save_txt=True,
    save_conf=True,
    save_crop=False,
    project=output_dir,
    name="test_run",
    exist_ok=True,
    verbose=True
)

print(f"\n Все предсказания сохранены в: {output_dir}/test_run")


image 1/19 /content/drive/MyDrive/prolog_500.v1i.yolov11/test/T=12.39.19 C=Side P=6183,7 A=91,2 L=0.png: 512x640 1 crack, 61.5ms
image 2/19 /content/drive/MyDrive/prolog_500.v1i.yolov11/test/T=12.39.20 C=Side P=1874,1 A=360,0 L=0.png: 512x640 1 joint, 9.6ms
image 3/19 /content/drive/MyDrive/prolog_500.v1i.yolov11/test/T=12.39.20 C=Side P=380,6 A=80,0 L=0.png: 512x640 1 joint, 9.9ms
image 4/19 /content/drive/MyDrive/prolog_500.v1i.yolov11/test/T=12.39.20 C=Side P=7879,1 A=79,9 L=0.png: 512x640 1 joint, 9.6ms
image 5/19 /content/drive/MyDrive/prolog_500.v1i.yolov11/test/T=12.39.22 C=Side P=6214,0 A=91,2 L=0.png: 512x640 1 crack, 11.3ms
image 6/19 /content/drive/MyDrive/prolog_500.v1i.yolov11/test/T=12.39.23 C=Side P=1874,3 A=39,8 L=0.png: 512x640 1 joint, 9.4ms
image 7/19 /content/drive/MyDrive/prolog_500.v1i.yolov11/test/T=12.39.23 C=Side P=380,6 A=120,1 L=0.png: 512x640 1 joint, 14.5ms
image 8/19 /content/drive/MyDrive/prolog_500.v1i.yolov11/test/T=12.39.25 C=Side P=2875.1 A=108.0 L=0